In [1]:
library(mice)
library(finalfit)
library(ggplot2)
library(dplyr)
library(stats)
library(miceadds)
library(missMethods)
library(smotefamily)
library(caret)
library(cowplot)
library(mlr)
library(fastDummies)
library(DMwR2)
library(mlmi)
library(ranger)
library(scales)
library(Hmisc)

Warning message:
"pakiet 'mice' został zbudowany w wersji R 4.3.2"

Dołączanie pakietu: 'mice'


Następujący obiekt został zakryty z 'package:stats':

    filter


Następujące obiekty zostały zakryte z 'package:base':

    cbind, rbind


Warning message:
"pakiet 'ggplot2' został zbudowany w wersji R 4.3.3"

Dołączanie pakietu: 'dplyr'


Następujące obiekty zostały zakryte z 'package:stats':

    filter, lag


Następujące obiekty zostały zakryte z 'package:base':

    intersect, setdiff, setequal, union


* miceadds 3.16-18 (2023-01-06 10:54:00)

Ładowanie wymaganego pakietu: lattice

Warning message:
"pakiet 'cowplot' został zbudowany w wersji R 4.3.2"
Warning message:
"pakiet 'mlr' został zbudowany w wersji R 4.3.2"
Ładowanie wymaganego pakietu: ParamHelpers

Warning message:
"pakiet 'ParamHelpers' został zbudowany w wersji R 4.3.2"
Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to t

In [2]:
create_zero_df <- function(num_rows, num_cols, colnames) {
  #' Create dataframe of zeros.
  
  df <- as.data.frame(matrix(0, nrow = num_rows, ncol = num_cols))
  names(df) <- colnames
  return(df)
}

In [3]:
test_impute_mice <- function(data, m, maxit, method, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with mice package.

    if (m > 1) {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE)
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- with(imp, glm(formula=as.formula(formula), family=family))
        tab <- summary(pool(fit), "all", conf.int = TRUE)
        tab <- tab[, c("term", "estimate", "2.5 %", "97.5 %")]
        tab <- as.data.frame(tab)
    }

    else {
        imp <- mice(data, method=method, m=m, maxit=maxit, printFlag=FALSE) %>% complete()
        imp[intcols] <- imp[intcols] %>% lapply(as.integer)
        fit <- glm(formula=formula, family=family, data=imp)

        coefs <- as.data.frame(fit$coefficients)
        colnames(coefs) <- c("estimate")
        coefs <- cbind(term=rownames(coefs), coefs)
        rownames(coefs) <- NULL

        conf <- confint(fit)
        rownames(conf) <- NULL
        
        tab <- cbind(coefs, conf)
    }
    
    return(tab)
}

In [4]:
simulate_mice <- function(nsim, data, m, maxit, method, formula, family, intcols) {
    #' Simulate statistical estimates computation for data imputed with mice package.

    tst_imp <- create_zero_df(6*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        start_index <- (n - 1) * 6 + 1
        end_index <- n * 6
        tst_imp_n <- test_impute_mice(data_n, m, maxit, method, formula, family, intcols)
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(tst_imp)
}

In [5]:
pool_mlmi_helper <- function(data, formula, family) {
    #' Helper function for pooling estimates after MLMI imputation.

    fit <- glm(formula=as.formula(formula), family=family, data=data)
    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    res <- cbind(coefs, conf)

    return(res)
}

In [6]:
pool_mlmi <- function(imp, M, formula, family) {
    #' Pool statistical estimates for data imputed with MLMI.

    data <- imp[[1]]
    res <- pool_mlmi_helper(data, formula, family)

    for (i in 2:M) {
        data_i <- imp[[i]]
        res_i <- pool_mlmi_helper(data_i, formula, family)
        res <- rbind(res, res_i)
    }

    pooled <- aggregate(res[, c(2, 3, 4)], list(res$term), FUN=mean)
    colnames(pooled)[1] <- "term"

    return(pooled)

}

In [7]:
test_impute_mlmi <- function(data, M, nCat, rseed, formula, family, bcols, fcols, intcols) {
    #' Compute statistical estimates for data imputed with MLMI.
    
    imp <- mixImp(obsData = data, nCat = nCat, M = M, rseed = rseed)
    
    for (i in 1:M) {

        for (bcol in bcols) {
            imp[[i]][bcol] <- ifelse(imp[[i]][bcol] == 1, 0, 1)
        }

        imp[[i]][fcols] <- imp[[i]][fcols] %>% lapply(factor)
        imp[[i]][intcols] <- imp[[i]][intcols] %>% lapply(as.integer)
    }
    
    pooled <- pool_mlmi(imp, M, formula, family)

    return(pooled)
}

In [8]:
simulate_mlmi <- function(nsim, data, M, nCat, formula, family, bcols, fcols, intcols) {
    #' Simulate statistical estimates computation for data imputed with MLMI.
    
    tst_imp <- create_zero_df(6*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        rseed <- sample(1:5000, 1)
        tst_imp_n <- test_impute_mlmi(data_n, M, nCat, rseed, formula, family, bcols, fcols, intcols)
        start_index <- (n - 1) * 6 + 1
        end_index <- n * 6
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
}

In [9]:
test_impute_knn <- function(data, formula, family, intcols) {
    #' Compute statistical estimates for data imputed with KNN.

    imp <- knnImputation(data, scale=FALSE)
    imp[intcols] <- imp[intcols] %>% lapply(as.integer)

    fit <- glm(formula=formula, family=family, data=imp)

    coefs <- as.data.frame(fit$coefficients)
    colnames(coefs) <- c("estimate")
    coefs <- cbind(term=rownames(coefs), coefs)
    rownames(coefs) <- NULL

    conf <- confint(fit)
    rownames(conf) <- NULL
    
    tab <- cbind(coefs, conf)

    return(tab)
}

In [10]:
simulate_knn <- function(nsim, data, formula, family, intcols) {
    #' Simulate statistical estimates computation for data imputed with KNN.

    tst_imp <- create_zero_df(6*nsim, 4, c("term", "estimate", "2.5 %", "97.5 %"))

    for (n in 1:nsim) {
        data_n <- sample_n(data, size=nrow(data), replace=TRUE)
        tst_imp_n <- test_impute_knn(data_n, formula, family, intcols)
        start_index <- (n - 1) * 6 + 1
        end_index <- n * 6
        tst_imp[start_index:end_index, ] <- tst_imp_n
    }

    return(as.data.frame(tst_imp))
    
}

In [11]:
PB <- function(sim_imp, real_coefs) {
    #' Compute percent bias.

    exp_est <- aggregate(sim_imp$estimate, list(sim_imp$term), FUN=mean)
    rownames(exp_est) <- exp_est$Group.1
    exp_est <- subset(exp_est, select=-1)

    terms <- unique(sim_imp$term)
    pbs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        pbs[t] <- 100*abs((exp_est[terms[t],] - as.numeric(real_coefs[terms[t]])) / as.numeric(real_coefs[terms[t]]))
    }
    
    return(pbs)
}

In [12]:
CR <- function(sim_imp, real_coefs) {
    #' Compute coverage rate.

    terms <- unique(sim_imp$term)
    crs <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        crs[t] <- mean(tsubs["2.5 %"] < as.numeric(real_coefs[terms[t]]) & as.numeric(real_coefs[terms[t]]) < tsubs["97.5 %"])
    }

    return(crs)
}

In [13]:
AW <- function(sim_imp, real_coefs) {
    #' Compute average width of confidence intervals.

    terms <- unique(sim_imp$term)
    aws <- rep(0, length(terms))

    for (t in 1:length(terms)) {
        tsubs <- subset(sim_imp, term==terms[t])
        diff <- tsubs["97.5 %"] - tsubs["2.5 %"]
        aws[t] <- mean(diff[, 1])
    }

    return(aws)   
}

In [14]:
fram <- read.csv("Framingham.csv")
fram_mar <- read.csv("fram_mar.csv")

In [15]:
fcols <- c("male", "education", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
bcols <- c("male", "currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes", "TenYearCHD")
intcols <- c("age", "cigsPerDay", "glucose", "totChol", "heartRate")
fram_mar[fcols] <- fram_mar[fcols] %>% lapply(factor)
fram[fcols] <- fram[fcols] %>% lapply(factor)

In [16]:
glm_fit <- glm(formula = TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose, family = "binomial", data = fram)

In [17]:
real_coefs <- glm_fit$coefficients
real_coefs

(Intercept)   cigsPerDay          BMI      totChol    heartRate      glucose 
-5.353487014  0.018861539  0.059448216  0.004655288 -0.003840925  0.012797899

##### Mean imputation

In [41]:
method1 <- rep("", ncol(fram_mar))
method1[c(5, 10, 13, 14, 15)] <- "mean"

In [42]:
sim_imp1 <- simulate_mice(1000, fram_mar, 1, 1, method1, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [43]:
pb1 <- PB(sim_imp1, real_coefs)
pb1

[1]  6.847271  9.195533  3.106686 38.156803  1.345943  9.066665

In [44]:
cr1 <- CR(sim_imp1, real_coefs)
cr1

[1] 0.920 0.953 0.941 0.761 0.955 0.905

In [45]:
aw1 <- AW(sim_imp1, real_coefs)
aw1

[1] 2.761880483 0.018714052 0.064138500 0.005882439 0.021550976 0.009855117

##### Linear regression imputation

In [46]:
method2 <- rep("", ncol(fram_mar))
method2[c(5, 10, 13, 14, 15)] <- "norm.predict"

In [47]:
sim_imp2 <- simulate_mice(1000, fram_mar, 1, 1, method2, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [48]:
pb2 <- PB(sim_imp2, real_coefs)
pb2

[1]   7.102282   6.683127  23.889906  11.069797 161.036641  29.388044

In [49]:
cr2 <- CR(sim_imp2, real_coefs)
cr2

[1] 0.841 0.917 0.774 0.864 0.731 0.648

In [50]:
aw2 <- AW(sim_imp2, real_coefs)
aw2

[1] 2.604586821 0.017933071 0.062471544 0.005731034 0.022023165 0.009680913

##### Stochastic regression imputation

In [51]:
method3 <- rep("", ncol(fram_mar))
method3[c(5, 10, 13, 14, 15)] <- "norm.nob"

In [52]:
sim_imp3 <- simulate_mice(1000, fram_mar, 1, 1, method3, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [53]:
pb3 <- PB(sim_imp3, real_coefs)
pb3

[1]  4.084134  5.044834  2.757093 18.109245 57.244187 16.698097

In [54]:
cr3 <- CR(sim_imp3, real_coefs)
cr3

[1] 0.881 0.913 0.872 0.852 0.849 0.788

In [55]:
aw3 <- AW(sim_imp3, real_coefs)
aw3

[1] 2.338268880 0.017388496 0.055265409 0.004992200 0.018909484 0.008847632

##### Predictive mean matching

In [56]:
method4 <- rep("", ncol(fram_mar))
method4[c(5, 10, 13, 14, 15)] <- "pmm"

In [57]:
sim_imp4 <- simulate_mice(1000, fram_mar, 1, 1, method4, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [58]:
pb4 <- PB(sim_imp4, real_coefs)
pb4

[1]  6.704995  5.985008  3.626334 20.879032 56.448974  7.744078

In [59]:
cr4 <- CR(sim_imp4, real_coefs)
cr4

[1] 0.849 0.930 0.873 0.819 0.830 0.848

In [60]:
aw4 <- AW(sim_imp4, real_coefs)
aw4

[1] 2.344811088 0.017398317 0.054724018 0.005057793 0.019092446 0.008555656

##### MICE with linear regression

In [61]:
method5 <- rep("", ncol(fram_mar))
method5[c(5, 10, 13, 14, 15)] <- "norm.predict"

In [62]:
sim_imp5 <- simulate_mice(1000, fram_mar, 10, 25, method5, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"


In [63]:
pb5 <- PB(sim_imp5, real_coefs)
pb5

[1]   5.244163  10.409642  27.113243  11.403853 226.034607  31.910702

In [64]:
cr5 <- CR(sim_imp5, real_coefs)
cr5

[1] 0.857 0.876 0.748 0.857 0.631 0.643

In [65]:
aw5 <- AW(sim_imp5, real_coefs)
aw5

[1] 2.575198844 0.017945783 0.062533186 0.005745928 0.022284288 0.009625363

##### MICE with stochastic regression

In [18]:
method6 <- rep("", ncol(fram_mar))
method6[c(5, 10, 13, 14, 15)] <- "norm.nob"

In [19]:
sim_imp6 <- simulate_mice(1000, fram_mar, 10, 25, method6, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Warning message:
"Number of logged events: 500"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"


In [20]:
pb6 <- PB(sim_imp6, real_coefs)
pb6

[1]  4.431492  3.375586  2.263447 18.059395 67.600894 16.393177

In [21]:
cr6 <- CR(sim_imp6, real_coefs)
cr6

[1] 0.929 0.948 0.927 0.902 0.915 0.850

In [22]:
aw6 <- AW(sim_imp6, real_coefs)
aw6

[1] 2.561789764 0.018427297 0.061936192 0.005423081 0.021997315 0.009387364

##### MICE with pmm

In [23]:
method7 <- rep("", ncol(fram_mar))
method7[c(5, 10, 13, 14, 15)] <- "pmm"

In [24]:
sim_imp7 <- simulate_mice(1000, fram_mar, 10, 25, method7, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Warning message:
"Number of logged events: 250"
Warning message:
"Number of logged events: 250"


In [25]:
pb7 <- PB(sim_imp7, real_coefs)
pb7

[1]  6.884011  6.951901  4.488530 20.444478 58.985663  8.551277

In [26]:
cr7 <- CR(sim_imp7, real_coefs)
cr7

[1] 0.917 0.949 0.958 0.908 0.946 0.937

In [27]:
aw7 <- AW(sim_imp7, real_coefs)
aw7

[1] 2.706384848 0.018351167 0.064979094 0.005742206 0.023803030 0.010220000

##### KNN imputation

In [28]:
sim_imp8 <- simulate_knn(1000, fram_mar, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [29]:
pb8 <- PB(sim_imp8, real_coefs)
pb8

[1]  5.8382852 11.4066433  9.6390648  0.7664263 19.8170212 19.0457229

In [30]:
cr8 <- CR(sim_imp8, real_coefs)
cr8

[1] 0.922 0.914 0.938 0.939 0.929 0.734

In [31]:
aw8 <- AW(sim_imp8, real_coefs)
aw8

[1] 2.407976324 0.017530751 0.055553832 0.005264121 0.019492498 0.008280411

##### MLMI

In [32]:
catfram <- fram_mar[, (names(fram_mar) %in% fcols)]
contfram <- fram_mar[, !(names(fram_mar) %in% colnames(catfram))]
fram_reorder <- cbind(catfram, contfram)
fram_reorder[] <- lapply(fram_reorder, function(x) if(is.factor(x)) as.integer(x) else x)

In [33]:
sim_imp9 <- simulate_mlmi(1000, fram_reorder, 10, 8, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", bcols, fcols, intcols)

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Imputing categorical variables using all 2-way associations.

Imputing continuous variables assuming main effects of categorical variables



Steps of ECM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...


Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...



In [34]:
pb9 <- PB(sim_imp9, real_coefs)
pb9

[1]  4.300872  2.099469  4.801326 18.507843 80.246700 15.795443

In [35]:
cr9 <- CR(sim_imp9, real_coefs)
cr9

[1] 0.910 0.904 0.926 0.780 0.849 0.870

In [36]:
aw9 <- AW(sim_imp9, real_coefs)
aw9

[1] 2.332071502 0.055497525 0.017415871 0.008916589 0.018969001 0.004994137

##### Decision tree imputation

In [37]:
method10 <- rep("", ncol(fram_mar))
method10[c(5, 10, 13, 14, 15)] <- "cart"

In [38]:
sim_imp10 <- simulate_mice(1000, fram_mar, 1, 1, method10, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Warning message:
"Number of logged events: 1"
Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profi

In [39]:
pb10 <- PB(sim_imp10, real_coefs)
pb10

[1]  0.784858 10.326083  2.910659 14.199604 39.696173  3.674072

In [40]:
cr10 <- CR(sim_imp10, real_coefs)
cr10

[1] 0.914 0.916 0.918 0.854 0.905 0.854

In [41]:
aw10 <- AW(sim_imp10, real_coefs)
aw10

[1] 2.325521407 0.017277082 0.053446389 0.005023341 0.018891777 0.008511692

##### Random forest imputation

In [42]:
method11 <- rep("", ncol(fram_mar))
method11[c(5, 10, 13, 14, 15)] <- "rf"

In [43]:
sim_imp11 <- simulate_mice(1000, fram_mar, 1, 1, method11, "TenYearCHD ~ cigsPerDay + BMI + totChol + heartRate + glucose", "binomial", intcols)

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowania...

Oczekiwanie na wykonanie profilowa

In [44]:
pb11 <- PB(sim_imp11, real_coefs)
pb11

[1]  3.590510 14.586845  5.237365 21.177785 41.399337  1.019951

In [45]:
cr11 <- CR(sim_imp11, real_coefs)
cr11

[1] 0.915 0.905 0.925 0.842 0.919 0.889

In [46]:
aw11 <- AW(sim_imp11, real_coefs)
aw11

[1] 2.329550075 0.017313323 0.053530396 0.005031375 0.018913675 0.008461414

##### Summary

In [4]:
pbs <- rbind(c("mean", pb1), c("linear regression", pb2), c("stochastic regression", pb3), c("pmm", pb4), c("MICE with linreg", pb5), 
c("MICE with stochreg", pb6), c("MICE with pmm", pb7), c("KNN", pb8), c("MLMI", pb9), c("decision tree", pb10), c("random forest", pb11))
pbs <- as.data.frame(pbs)
colnames(pbs)[2:7] <- names(real_coefs)
pbs[,2:7] <- pbs[,2:7] %>% lapply(as.numeric)
pbs$mean_pb <- rowMeans(pbs[,2:7])
pbs$total_pb <- rowSums(pbs[,2:7])
pbs

V1,X.Intercept.,cigsperDay,BMI,totChol,heartRate,glucose,mean_PB,total_PB
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mean,6.847271,9.195533,3.106686,38.1568028,1.345943,9.066665,11.28648,67.71890
linear regression,7.102282,6.683127,23.889906,11.0697973,161.036641,29.388044,39.86163,239.16980
stochastic regression,4.084134,5.044834,2.757093,18.1092452,57.244187,16.698097,17.32293,103.93759
pmm,6.704995,5.985008,3.626334,20.8790319,56.448974,7.744078,16.89807,101.38842
MICE with linreg,5.244163,10.409642,27.113243,11.4038530,226.034607,31.910702,52.01937,312.11621
MICE with stochreg,4.431492,3.375586,2.263447,18.0593955,67.600894,16.393177,18.68733,112.12399
MICE with PMM,6.884011,6.951901,4.488530,20.4444782,58.985663,8.551277,17.71764,106.30586
KNN,5.838285,11.406643,9.639065,0.7664263,19.817021,19.045723,11.08553,66.51316
MLMI,4.300872,2.099469,4.801326,18.5078432,80.246700,15.795443,20.95861,125.75165


In [5]:
crs <- rbind(c("mean", cr1), c("linear regression", cr2), c("stochastic regression", cr3), c("pmm", cr4), c("MICE with linreg", cr5), 
c("MICE with stochreg", cr6), c("MICE with pmm", cr7), c("KNN", cr8), c("MLMI", cr9), c("decision tree", cr10), c("random forest", cr11))
crs <- as.data.frame(crs)
colnames(crs)[2:7] <- names(real_coefs)
crs[,2:7] <- crs[,2:7] %>% lapply(as.numeric)
crs$mean_cr <- rowMeans(crs[,2:7])
crs$total_cr <- rowSums(crs[,2:7])
crs

V1,X.Intercept.,cigsperDay,BMI,totChol,heartRate,glucose,mean_CR,total_CR
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mean,0.920,0.953,0.941,0.761,0.955,0.905,0.9058333,5.435
linear regression,0.841,0.917,0.774,0.864,0.731,0.648,0.7958333,4.775
stochastic regression,0.881,0.913,0.872,0.852,0.849,0.788,0.8591667,5.155
pmm,0.849,0.930,0.873,0.819,0.830,0.848,0.8581667,5.149
MICE with linreg,0.857,0.876,0.748,0.857,0.631,0.643,0.7686667,4.612
MICE with stochreg,0.929,0.948,0.927,0.902,0.915,0.850,0.9118333,5.471
MICE with PMM,0.917,0.949,0.958,0.908,0.946,0.937,0.9358333,5.615
KNN,0.922,0.914,0.938,0.939,0.929,0.734,0.8960000,5.376
MLMI,0.910,0.904,0.926,0.780,0.849,0.870,0.8731667,5.239


In [6]:
aws <- rbind(c("mean", aw1), c("linear regression", aw2), c("stochastic regression", aw3), c("pmm", aw4), c("MICE with linreg", aw5), 
c("MICE with stochreg", aw6), c("MICE with pmm", aw7), c("KNN", aw8), c("MLMI", aw9), c("decision tree", aw10), c("random forest", aw11))
aws <- as.data.frame(aws)
colnames(aws)[2:7] <- names(real_coefs)
aws[,2:7] <- aws[,2:7] %>% lapply(as.numeric)
aws$mean_aw <- rowMeans(aws[,2:7])
aws$total_aw <- rowSums(aws[,2:7])
aws

V1,X.Intercept.,cigsperDay,BMI,totChol,heartRate,glucose,mean_AW,total_AW
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mean,2.761880,0.01871405,0.06413850,0.005882439,0.02155098,0.009855117,0.4803369,2.882022
linear regression,2.604587,0.01793307,0.06247154,0.005731034,0.02202317,0.009680913,0.4537378,2.722427
stochastic regression,2.338269,0.01738850,0.05526541,0.004992200,0.01890948,0.008847632,0.4072787,2.443672
pmm,2.344811,0.01739832,0.05472402,0.005057793,0.01909245,0.008555656,0.4082732,2.449639
MICE with linreg,2.575199,0.01794578,0.06253319,0.005745928,0.02228429,0.009625363,0.4488889,2.693333
MICE with stochreg,2.561790,0.01842730,0.06193619,0.005423081,0.02199731,0.009387364,0.4464935,2.678961
MICE with PMM,2.706385,0.01835117,0.06497909,0.005742206,0.02380303,0.010220000,0.4715801,2.829480
KNN,2.407976,0.01753075,0.05555383,0.005264121,0.01949250,0.008280411,0.4190163,2.514098
MLMI,2.332072,0.05549752,0.01741587,0.008916589,0.01896900,0.004994137,0.4063108,2.437865


In [ ]:
+